In [1]:
import sys
sys.path.append('../')

import numpy as np
import matplotlib.pyplot as plt
import uproot as up

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
from scripts.optimal_filter.collect_data import *


In [3]:
filename = '../../../../offline-samples/Baseline2019_bb0n_center_seed82_comsol.nEXOevents.root'
temp = up.open(filename)
temp_noise_tag = temp['Event/Elec/ElecEvent/ElecEvent/fElecChannels/fElecChannels.fChannelNoiseTag'].array(library='np')
event_count = temp_noise_tag.shape[0]
del temp
del temp_noise_tag

In [4]:
reconstruct = np.zeros(event_count)
error_limit = np.zeros(event_count)

In [5]:
from tqdm.notebook import tqdm

#for e in tqdm(range(event_count)):
for e in tqdm(range(1)):
    data = event_data(filename, e)
    data.get_signal()
    truth = data.signal
    
    data.get_coordinates()
    X, Y, Z = data.coordinates
    
    data.get_noise_tag()
    noise_tag = data.noise_tag
    
    data.get_channel_electrons()
    electrons = data.channel_electrons
    
    data.get_event_electrons()
    data.get_E()
    data.get_lineage()
    
    Z_anode = 403
    Z_cathode = 1586
    R_rings = 556
    fiducial_cut = 20
    
    if np.sqrt(X**2+Y**2) > R_rings-fiducial_cut or abs(Z) < Z_anode+fiducial_cut or abs(Z) > Z_cathode-fiducial_cut:
      print('Event outside of fiducial volume')
      reconstruct[e] = np.nan
      continue

      
    N = len(truth[0])
    noise_traces = get_noise_new(10000)[0:N, :]
    noise_FFT = np.abs(np.fft.rfft(noise_traces, axis = 1))**2
    Fs = 2e6
    norm = 2/(N*Fs)
    J = np.mean(noise_FFT, axis = 0)*norm
    
    found_electrons = event_analysis(truth, J, Z, noise_tag, electrons, 400)

    reconstruct[e] = np.sum(found_electrons)

    print('\n')
    print('-'*20)
    print('Reconstruction error for this event: ', reconstruct[e])
    print('-'*20)
    print('\n')
    

  0%|          | 0/1 [00:00<?, ?it/s]

------------------------------
Channel number:  0
Noise channel
------------------------------
Channel number:  1
Noise channel
------------------------------
Channel number:  2
This channel collected 541 electrons
But the integral of the true WVFM is 1619.3258056640625
Reconstructed amplitude is 472.87231500247475
True amplitude of the signal is 474.0791931152344
Area of the unit pulse stencil is 3.4157280921936035
The correction factor for this event is 1.0368983674505128
------------------------------
Channel number:  3
This channel collected 450 electrons
But the integral of the true WVFM is 1342.138671875
Reconstructed amplitude is 443.93024918101696
True amplitude of the signal is 446.77435302734375
Area of the unit pulse stencil is 3.0040650367736816
The correction factor for this event is 1.0368983674505128
------------------------------
Channel number:  4
Ion channel - no true signal observed.
------------------------------
Channel number:  5
Ion channel - no true signal obser